In [99]:
!pip install fake_useragent

In [100]:
!pip install pymorphy2

Defaulting to user installation because normal site-packages is not writeable


## Базовые требования к домашкам

1. Формат - jupyter-тетрадка или скрипт на питоне
2. Мы запускаем ваши тетрадки с нуля, поэтому следите, чтобы не было 
- необъявленных переменных (удалили ячейку, а переменная продолжает использоваться)
- лишних принтов отладочной информации (пожалуйста! это очень мешает проверяющему, когда приходится пролистать 10 страниц текста без кода)
3. Комментарии приветствуются!

## Задание: Оценка тональности по словарю
В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является отзыв положительным или отрицательным. 
Делать мы будем это таким образом: мы возьмём некоторое число отзывов, заранее размеченных как положительные или отрицательные; выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов  в поступившем нам на проверку отзыве больше.

Мы будем работать по заранее определённому пайплайну:

1. Сначала нам надо скачать данные -- соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления "тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. (2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга)
Примечание: сбор данных с помощью краулинга может занять много времени, советуем сначала реализовать всё задание на готовых данных, а затем сделать с краулингом, если хотите получить 9 или 10.

In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd

Возьму отзывы на драмы с сайта kinopoisk.ru. Всего 80 положительных, 80 отрицательных и 20 валидационных.

Вариант с краулингом сайта, который иногда не работает из-за блокировок со стороны сайта.

In [2]:
good_url = 'https://www.kinopoisk.ru/reviews/type/comment/status/good/genre/8/period/month/perpage/100/#list'
bad_url = 'https://www.kinopoisk.ru/reviews/type/comment/status/bad/genre/8/period/month/perpage/100/#list'

In [3]:
user_agent = UserAgent().chrome
good_response = requests.get(good_url, headers={'User-Agent': user_agent})
good_response.encoding = 'utf-8'
good_page = good_response.text
good_soup = BeautifulSoup(good_page, 'html.parser')

In [4]:
user_agent = UserAgent().chrome
bad_response = requests.get(bad_url, headers={'User-Agent': user_agent})
bad_response.encoding = 'utf-8'
bad_page = bad_response.text
bad_soup = BeautifulSoup(bad_page, 'html.parser')

Вариант с парсингом скачанной html-страницы.

good_url = https://www.kinopoisk.ru/reviews/type/comment/status/good/genre/8/period/month/perpage/100/#list

bad_url = https://www.kinopoisk.ru/reviews/type/comment/status/bad/genre/8/period/month/perpage/100/#list

In [11]:
with open('good_page_100.html', encoding='utf-8') as good_page:
    good_soup = BeautifulSoup(good_page, 'html.parser')
with open('bad_page_100.html', encoding='utf-8') as bad_page:
    bad_soup = BeautifulSoup(bad_page, 'html.parser')

-------------

In [12]:
good_reviews = good_soup.find_all('span', {'itemprop': 'reviewBody'})
bad_reviews = bad_soup.find_all('span', {'itemprop': 'reviewBody'})

In [13]:
def extract_text(reviews):
    review_texts = []
    for review in reviews:
        review_texts.append(review.text)
    return review_texts

In [14]:
good_review_texts = extract_text(good_reviews)
bad_review_texts = extract_text(bad_reviews)

In [15]:
good_df = pd.DataFrame(good_review_texts)
good_df['label'] = 1
good_df = good_df.rename(columns={0: 'review'})
good_validate = good_df.tail(20)
good_df = good_df.head(80)

In [16]:
bad_df = pd.DataFrame(bad_review_texts)
bad_df['label'] = 0
bad_df = bad_df.rename(columns={0: 'review'})
bad_validate = bad_df.tail(20)
bad_df = bad_df.head(80)

In [17]:
X = pd.concat([good_df, bad_df], ignore_index=True)
X.sample(5)

,review,label
112,"Трудно назвать этот сериал «выдающимся», так к...",0
99,Испанский стыд это единственное словосочетание...,0
22,Создателями сериала являются Баран бо Одар (ре...,1
150,Это мой первый фильм про стрельбу в школе. «Бу...,0
95,"Данный фильм относится к категории, которая в ...",0


In [18]:
validate = pd.concat([good_validate, bad_validate])
y = validate.sample(20, ignore_index=True)
y.sample(5)

,review,label
18,Фильм Тарковского — один из ярчайших примеров ...,0
12,Фильм слепленный на коленке. Попытка отрефлекс...,0
13,Что за дрянь я посмотрел. Нудятина в обертке д...,0
11,"В фильме затронута крайне важная тема, но к со...",0
15,"Предупреждаем, что ту самую книгу Агаты Кристи...",1


2. Токенизируйте слова, приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

In [19]:
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import RegexpTokenizer

In [20]:
morph = MorphAnalyzer()

def lemmatize(review):
    tokens = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for token in tokenizer.tokenize(review):
        token = morph.normal_forms(token)[0]
        tokens = tokens + ' ' + token
    return tokens

In [21]:
X_lemmatized = X.copy()
for i in range(len(X)):
    X_lemmatized.loc[i, 'review'] = lemmatize(X.loc[i, 'review'])

In [22]:
X_lemmatized.head()

,review,label
0,художественный фильм по мотив известный реаль...,1
1,у я просто язык не повернуться назвать это ка...,1
2,любовь всё преодолевать это очень противный о...,1
3,сериал клан сопрано the sopranos считаться од...,1
4,в 2006 год один из самый хороший и многообеща...,1


In [23]:
y_lemmatized = y.copy()
for i in range(len(y)):
    y_lemmatized.loc[i, 'review'] = lemmatize(y.loc[i, 'review'])

In [24]:
y_lemmatized.head()

,review,label
0,вступление выход крид наследие рокк в 2015 м ...,0
1,воскликнуть в искренний порыв ипполит в ирони...,0
2,наверняка быть тот кто пойти на этот фильм в ...,0
3,через два год после прочтение отличный книга ...,1
4,первый сезон зацепить особо неожиданный сюжет...,0


In [25]:
good_lemmatized = X_lemmatized[X_lemmatized['label'] == 1]
good_lemmatized.reset_index(inplace=True)
bad_lemmatized = X_lemmatized[X_lemmatized['label'] == 0]
bad_lemmatized.reset_index(inplace=True)

good_words_list = []
bad_words_list = []
for i in range(len(good_lemmatized)):
    good_words_list.extend(good_lemmatized.loc[i, 'review'].split())
    bad_words_list.extend(bad_lemmatized.loc[i, 'review'].split())

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (2 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)
В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачтём полный балл

In [26]:
from collections import Counter

In [27]:
good_cnt = Counter(good_words_list)
for key, value in good_cnt.copy().items():
    if value <= 3:
        del good_cnt[key]

In [28]:
bad_cnt = Counter(bad_words_list)
for key, value in bad_cnt.copy().items():
    if value <= 2:
        del bad_cnt[key]

In [29]:
only_good = set(good_cnt) - set(bad_cnt)
len(only_good)

441

In [30]:
only_bad = set(bad_cnt) - set(good_cnt)
len(only_bad)

540

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

In [31]:
def determine_review_tonality(review):
    review_lemmas = review.split()
    good_score = 0
    bad_score = 0
    for lemma in review_lemmas:
        if lemma in only_good:
            good_score += 1
        elif lemma in only_bad:
            bad_score += 1
    if good_score > bad_score:
        return 1
    else:
        return 0

In [32]:
accuracy = 0
for i in range(len(y_lemmatized)):
    review = y_lemmatized.loc[i, 'review']
    y_pred = determine_review_tonality(review)
    y_true = y_lemmatized.loc[i, 'label']
    if y_pred == y_true:
        accuracy += 1
accuracy = accuracy / len(y_lemmatized)
accuracy

0.7

5. Предложите как минимум 2 способа улучшить этот алгоритм определения тональности отзыва (1 балл за описание и реализацию каждого способа; если 2 способа описаны только текстом, это 1 балл. За третий и последующие способы дополнительных баллов не будет)

**Вариант 1.** Можно работать не с множествами уникальных для отзывов данной тональности слов, а со словарями частотности слов в положительных и в отрицательных отзывах и давать балл в пользу той тональности, в которой это слово встречается чаще (на всякий случай - значительно чаще, по крайней мере на 5 раз чаще).

In [33]:
def determine_review_tonality_better1(review):
    review_lemmas = review.split()
    good_score = 0
    bad_score = 0
    for lemma in review_lemmas:
        if lemma in good_cnt and lemma not in bad_cnt:
            good_score += 1
        elif lemma in bad_cnt and lemma not in good_cnt:
            bad_score += 1
        elif lemma in good_cnt and lemma in bad_cnt:
            if good_cnt[lemma] >= bad_cnt[lemma] + 5:
                good_score += 1
            elif bad_cnt[lemma] >= good_cnt[lemma] + 5:
                bad_score += 1
    if good_score > bad_score:
        return 1
    else:
        return 0

In [34]:
accuracy = 0
for i in range(len(y_lemmatized)):
    review = y_lemmatized.loc[i, 'review']
    y_pred = determine_review_tonality_better1(review)
    y_true = y_lemmatized.loc[i, 'label']
    if y_pred == y_true:
        accuracy += 1
accuracy = accuracy / len(y_lemmatized)
accuracy

0.5

Как видим, метод оказался весьма так себе - качество он скорее портит, чем улучшает. Видимо, это связано с тем, что в моих словарях положительных и отрицательных слов, даже несмотря на удаление слишком низкочастотных, очень много случайных слов и оттого работа просто с частотностями тоже оказывается во многом случайно раздающей баллы.

**Вариант 2.** Можно попробовать поработать со стоп-словами, исключив их из списков слов для тональностей.

In [35]:
from nltk.corpus import stopwords

In [36]:
russian_stopwords = stopwords.words("russian")

In [37]:
len(only_good), len(only_bad)

(441, 540)

In [38]:
def remove_stopwords(only_words):
    only_words
    for only_word in only_words.copy():
        if only_word in russian_stopwords:
            only_words.remove(only_word)

In [39]:
remove_stopwords(only_good)
remove_stopwords(only_bad)

In [40]:
len(only_good), len(only_bad)

(440, 538)

In [41]:
accuracy = 0
for i in range(len(y_lemmatized)):
    review = y_lemmatized.loc[i, 'review']
    y_pred = determine_review_tonality(review)
    y_true = y_lemmatized.loc[i, 'label']
    if y_pred == y_true:
        accuracy += 1
accuracy = accuracy / len(y_lemmatized)
accuracy

0.65

Как видим, стоп-слов оказалось не настолько много и они оказались не настолько значимыми, чтобы повлиять на качество определения тональности. Либо же качество тоже ухудшается скорее в меньшую сторону, видимо, оттого, что всё-таки при определении тональности нередко значимыми оказываются как раз высокочастотные слова (хотя это не всегда так и в задачах определения тональности, насколько я знаю, стоп-слова иногда действительно удаляются, и это улучшает качество).